## 2D Fourier Neural Network for SABR Model
Inputs: Boundary conditions C(S,T,alpha) = max(S-K,0). 1024 realizations for random $K \in [0,1]$ and 241*241 grid points for $S \in [0.1,1]$ and $\alpha in [0.1,1]$

Outputs: European Call option prices from SABR(Hogan2002) model at $t=0$


Data generated in data_generation/SABR


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

import matplotlib.pyplot as plt

import operator
from functools import reduce
from functools import partial

from timeit import default_timer
from utilities3 import *

torch.manual_seed(0)
np.random.seed(0)


################################################################
# fourier layer
################################################################
class SpectralConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1, modes2):
        super(SpectralConv2d, self).__init__()

        """
        2D Fourier layer. It does FFT, linear transform, and Inverse FFT.
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1 #Number of Fourier modes to multiply, at most floor(N/2) + 1
        self.modes2 = modes2

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, dtype=torch.cfloat))
        self.weights2 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul2d(self, input, weights):
        # (batch, in_channel, x,y ), (in_channel, out_channel, x,y) -> (batch, out_channel, x,y)
        return torch.einsum("bixy,ioxy->boxy", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        #Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.rfft2(x)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels,  x.size(-2), x.size(-1)//2 + 1, dtype=torch.cfloat, device=x.device)
        out_ft[:, :, :self.modes1, :self.modes2] = \
            self.compl_mul2d(x_ft[:, :, :self.modes1, :self.modes2], self.weights1)
        out_ft[:, :, -self.modes1:, :self.modes2] = \
            self.compl_mul2d(x_ft[:, :, -self.modes1:, :self.modes2], self.weights2)

        #Return to physical space
        x = torch.fft.irfft2(out_ft, s=(x.size(-2), x.size(-1)))
        return x

class FNO2d(nn.Module):
    def __init__(self, modes1, modes2,  width):
        super(FNO2d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .

        input: the solution of the coefficient function and locations (a(x, y), x, y)
        input shape: (batchsize, x=s, y=s, c=3)
        output: the solution
        output shape: (batchsize, x=s, y=s, c=1)
        """

        self.modes1 = modes1
        self.modes2 = modes2
        self.width = width
        self.fc0 = nn.Linear(3, self.width) # input channel is 3: (a(x, y), x, y)

        self.conv0 = SpectralConv2d(self.width, self.width, self.modes1, self.modes2)
        self.conv1 = SpectralConv2d(self.width, self.width, self.modes1, self.modes2)
        self.conv2 = SpectralConv2d(self.width, self.width, self.modes1, self.modes2)
        self.conv3 = SpectralConv2d(self.width, self.width, self.modes1, self.modes2)
        self.w0 = nn.Conv1d(self.width, self.width, 1)
        self.w1 = nn.Conv1d(self.width, self.width, 1)
        self.w2 = nn.Conv1d(self.width, self.width, 1)
        self.w3 = nn.Conv1d(self.width, self.width, 1)


        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        batchsize = x.shape[0]
        size_x, size_y = x.shape[1], x.shape[2]

        x = self.fc0(x)
        x = x.permute(0, 3, 1, 2)

        x1 = self.conv0(x)
        x2 = self.w0(x.view(batchsize, self.width, -1)).view(batchsize, self.width, size_x, size_y)
        x = x1 + x2
        x = F.relu(x)

        x1 = self.conv1(x)
        x2 = self.w1(x.view(batchsize, self.width, -1)).view(batchsize, self.width, size_x, size_y)
        x = x1 + x2
        x = F.relu(x)

        x1 = self.conv2(x)
        x2 = self.w2(x.view(batchsize, self.width, -1)).view(batchsize, self.width, size_x, size_y)
        x = x1 + x2
        x = F.relu(x)

        x1 = self.conv3(x)
        x2 = self.w3(x.view(batchsize, self.width, -1)).view(batchsize, self.width, size_x, size_y)
        x = x1 + x2

        x = x.permute(0, 2, 3, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [2]:
################################################################
# configs
################################################################
TRAIN_PATH = 'data/SABR.mat'
TEST_PATH = 'data/SABR.mat'

ntrain = 900
ntest = 100

batch_size = 20
learning_rate = 0.001

epochs = 20
step_size = 100
gamma = 0.5

modes = 12
width = 32

r = 2
h = int(((241 - 1)/r) + 1)
s = h

################################################################
# load data and data normalization
################################################################
reader = MatReader(TRAIN_PATH)
x_train = reader.read_field('a')[:ntrain,::r,::r][:,:s,:s]
y_train = reader.read_field('u')[:ntrain,::r,::r][:,:s,:s]

reader.load_file(TEST_PATH)
x_test = reader.read_field('a')[:ntest,::r,::r][:,:s,:s]
y_test = reader.read_field('u')[:ntest,::r,::r][:,:s,:s]

x_normalizer = UnitGaussianNormalizer(x_train)
x_train = x_normalizer.encode(x_train)
x_test = x_normalizer.encode(x_test)

y_normalizer = UnitGaussianNormalizer(y_train)
y_train = y_normalizer.encode(y_train)

grids = []
grids.append(np.linspace(0.1, 1, s))
grids.append(np.linspace(0.1, 1, s))
grid = np.vstack([xx.ravel() for xx in np.meshgrid(*grids)]).T
grid = grid.reshape(1,s,s,2)
grid = torch.tensor(grid, dtype=torch.float)
x_train = torch.cat([x_train.reshape(ntrain,s,s,1), grid.repeat(ntrain,1,1,1)], dim=3)
x_test = torch.cat([x_test.reshape(ntest,s,s,1), grid.repeat(ntest,1,1,1)], dim=3)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

In [3]:
################################################################
# training and evaluation
################################################################
model = FNO2d(modes, modes, width).cuda()
print(count_params(model))

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = LpLoss(size_average=False)
y_normalizer.cuda()
for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2 = 0
    train_mse = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()

        optimizer.zero_grad()
        out = model(x).reshape(batch_size, s, s)
        out = y_normalizer.decode(out)
        y = y_normalizer.decode(y)

        loss = myloss(out.view(batch_size,-1), y.view(batch_size,-1))
        loss.backward()

        mse = F.mse_loss(out.view(batch_size, -1), y.view(batch_size, -1), reduction='mean')

        optimizer.step()
        train_l2 += loss.item()
        train_mse += mse.item()

    scheduler.step()

    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.cuda(), y.cuda()

            out = model(x).reshape(batch_size, s, s)
            out = y_normalizer.decode(out)

            test_l2 += myloss(out.view(batch_size,-1), y.view(batch_size,-1)).item()

    train_mse /= len(train_loader)
    train_l2/= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(ep, t2-t1, train_mse,train_l2, test_l2)

1188353
0 3.628322899999999 0.01511332023009244 0.27258768545256723 0.06426535010337829
1 2.5643138999999984 0.0003219275861435259 0.053686861197153726 0.050306300520896914
2 2.6155501999999977 7.99526283824687e-05 0.02772614041964213 0.020022254288196564
3 2.5710431000000007 4.425264188385982e-05 0.020537407696247102 0.013677326142787933
4 2.601163400000001 2.4842086339857715e-05 0.014796157595184115 0.018074369728565215
5 2.6066271000000008 3.28490159517969e-05 0.018765546795394685 0.020627862215042113
6 2.6063176000000006 3.854566417026541e-05 0.019298221468925476 0.014581766426563263
7 2.6023257000000015 1.668156414274967e-05 0.012914923859967125 0.00865947589278221
8 2.442070700000002 2.314388824136889e-05 0.014852851397461361 0.01252780795097351
9 2.374375800000003 2.2916571520504983e-05 0.014763674951261943 0.009772391468286514
10 2.5666797000000017 1.670698236719343e-05 0.012390798462761773 0.007506652623414993
11 2.611481699999999 2.1573667961800107e-05 0.01349302678472466 0.0

In [8]:
# torch.save(model, 'model/ns_fourier_burgers')
pred = torch.zeros(y_test.shape)
benchmark = torch.zeros(y_test.shape)
inputs = torch.zeros(x_test.shape)
index = 0
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=1, shuffle=False)
with torch.no_grad():
    for x, y in test_loader:
        test_l2 = 0
        x, y = x.cuda(), y.cuda()

        out = model(x)

        pred[index] = out.squeeze()
        benchmark[index] = y.squeeze()
        inputs[index] = x

        test_l2 += myloss(out.view(1, -1), y.view(1, -1)).item()
        print(index, test_l2)
        index = index + 1

    scipy.io.savemat('pred/SABR_test.mat', mdict={'pred': pred.cpu().numpy()})
    scipy.io.savemat('pred/SABR_benchmark.mat', mdict={'benchmark': benchmark.cpu().numpy()})
    scipy.io.savemat('pred/SABR_inputs.mat', mdict={'inputs': inputs.cpu().numpy()})

0 2.406733512878418
1 1.687555193901062
2 1.062829852104187
3 1.7689999341964722
4 7.750137805938721
5 1.4297990798950195
6 2.976606845855713
7 0.6050462126731873
8 7.052504539489746
9 2.2151689529418945
10 0.8918940424919128
11 6.4709601402282715
12 2.049544095993042
13 2.4593350887298584
14 3.7202224731445312
15 0.8555641174316406
16 3.9725632667541504
17 1.3333805799484253
18 2.2414066791534424
19 0.4567543864250183
20 0.504222571849823
21 6.934656143188477
22 3.0560178756713867
23 0.5792312622070312
24 2.057722330093384
25 0.761605441570282
26 6.458011150360107
27 2.721839189529419
28 8.84695053100586
29 2.474086284637451
30 2.8018035888671875
31 1.0869773626327515
32 1.5830386877059937
33 5.941612720489502
34 0.4662610590457916
35 3.088047981262207
36 2.031775951385498
37 2.2449707984924316
38 1.4254095554351807
39 1.3870134353637695
40 7.325973033905029
41 0.5908007025718689
42 1.9955098628997803
43 8.506818771362305
44 5.230462074279785
45 1.9473047256469727
46 0.430556237697601

In [ ]:
pred = scipy.io.loadmat('pred/SABR_test.mat')
pred = pred['pred']

inputs = scipy.io.loadmat('pred/SABR_inputs.mat')
inputs = inputs['inputs']

benchmark = scipy.io.loadmat('pred/SABR_benchmark.mat')
benchmark = benchmark['benchmark']

xx = inputs[1,:,1]

for k in range(1,10,2):
    plt.figure()
    plt.plot(xx,pred[k,:],'--',label = 'predicted',linewidth=3)
    plt.plot(xx,benchmark[k,:],label = 'ground truth(BS)')
    plt.plot(xx,inputs[k,:,0],label='input(C(S,T))')
    plt.xlabel('S')
    plt.ylabel('C(S,0)')
    plt.legend()
    plt.show()
